# Titanic ML: Primera Aproximación

En este Notebook vamos a dejar preparado el código y las ideas necesarias para obtener una primera solución cerrada para el problema de predecir la supervivencia del Titanic. Después de esto esperamos haber tratado los principales aspectos, manteniendo abiertos qué puntos se pueden mejorar para que cada equipo pueda trabajar sobre ello.

### Primer Paso: Cargamos los Datos en tablas

Utilizaremos una librería muy útil de python, con varias funciones interesantes que ya vienen implementadas: Pandas.

Tenemos que cargar 2 datasets, uno que nos servirá para entrenar nuestro modelo, que contendrá la variable que queremos predecir; y otro que servirá para probar el modelo y cuyos resultados tendremos que subir a la competición Kaggle.

In [ ]:
# Importamos librerias necesarias
import pandas as pd, numpy as np

path = 'C:/Users/pere.miquel/Desktop/DATATHON_KAGGLE/'

# Cargamos los dos sets de datos en dos Dataframes de Pandas.
test_df = pd.read_csv(path + 'test.csv', sep = ',')
train_df = pd.read_csv(path + 'train.csv', sep = ',')

## Exploración y limpieza de los Datos

Ahora es el momento de movernos un poco por los datasets para familarizarnos con ellos. Normalmente, el testing dataset (usado para probar el modelo) tendrá una columna menos que el training dataset (el usado para entrenar el modelo), que será nuestra variable/columna Objectivo (o Target). Vamos a ver cuál es en nuestro caso:

In [ ]:
# Obtenemos la lista de nombres de columna en el training y testing datasets
tr_c = train_df.columns
te_c = test_df.columns

target = set(tr_c) - set(te_c)

print 'Las columnas presentes en ambos datasets son: %s \n' % (', '.join(tr_c))
print 'Nuestra(s) variable(s) objetivo es: %s' % (', '.join(target))

A la hora de entrenar un modelo de ML, hay que tener en cuenta dos cosas principalmente: 

1. Hay que diferenciar entre **features**, características o variables del modelo (las que se usan para predecir) y **target variable** que sería la variable a predecir

2. Nuestras *features* pueden ser de dos tipos: **Numéricas**, **Categóricas** o **Ordinales**. Diferencias entre estas variables:
 2. **Categóricas**: Tienen varias categorías (como su nombre indica) y cada  valor de la variable está dentro de una de estas categorías. No tiene sentido calcular medias o máximos/mínimos con ellas, pero sí calcular la frecuencia de aparición de cada categoría (por ejemplo)
 2. **Numéricas**: Pueden ser *continuas* o *discretas*. Toman valores numéricos entre un máximo y un mínimo. Con ellas tiene sentido calcular máximos, mínimos, medias... 
 2. **Ordinales**: Son similares a las variables categóricas, pero tienen un orden relativo entre ellas. Suelen tomar valores numéricos discretos (como su nombre indica)
    
Es el momento de visualizar los datos, para ver la pinta que tienen y sacar algunas conclusiones. Tenemos dos maneras de ver nuestros *Dataframes*: el método *sample(lineas_a_mostrar)*, o el método *head(lineas_a_mostrar)*. Usaremos el método sample() ya que la visualización es un poco mejor y comentaremos el otro.

In [ ]:
print train_df.dtypes

train_df.sample(n=5)

De esta visualización podemos sacar algunas conclusiones:
* **Features Numéricas**: Age (continua), Fare(continua), SibSp(Discrete), Parch(Discrete)
* **Features Categóricas**: Survived, Sex, Embarked, Pclass. También podriamos incluir Ticket y Cabin, aunque realmente no tienen pinta de ser categorías si no un *carácter alfanumérico* (viendo esto podemos empezar a suponer que no son demasiado significativas para nosotros, pero lo veremos más adelante)

También nos hacemos una idea del tipo que son cada una de nuestras variables:
* Survived: int
* Pclass: int 
* Name: string
* Sex: string
* Age: float
* SibSp: int
* Parch: int
* Ticket: string
* Fare: float
* Cabin: string
* Embarked: string

También podemos llevar a cabo un análisis estadístico de nuestros datasets con el comando usado previamente:

In [ ]:
print 'El número de pasajeros en nuestro Training Dataset es de: %s \n' % (train_df.shape[0])

train_df.describe(include = "all")

Gracias a este resumen estadístico, podemos sacar varias conclusiones:

* Tenemos un total de 891 pasajeros en nuestro training dataset.
* Hay algunas variables que no están informadas para todos los pasajeros:
    * **Age**: $\frac{(891-714)}{891} \times 100 = 19.8$% de campos no informados
    * **Cabin**: $\frac{(891-204)}{891} \times 100 = 77.1$% de campos no informados
    * **Embarked**: $\frac{(891-889)}{891} \times 100 = 0.22$% de campos no informados
 
Como hemos visto, nos hemos centrado sobretodo en investigar los valores nulos. ¿Por qué? Sencillamente porque son los valores que más problemas dan a la hora de entrenar un modelo de ML. La mejor manera de ver los valores nulos es comprobar si són nulos y contar los que lo sean usando pandas:

In [ ]:
print 'Número de nulos por campo en la dataframe de entrenamiento \n'
print pd.isnull(train_df).sum(), '\n'
print 'Número de nulos por campo en la dataframe de test \n'
print test_df.isnull().sum()

Hay muchas formas de tratar con los valores nulos en los datos. Hay modelos que ya tienen mecanismos internos para convivir con ellos, pero siempre es mejor aplicar alguna norma razonada.

Si nos fijamos en los valores faltantes del training dataset, nos encontramos 3 escenarios distintos: faltan muy pocos valores (Embarked), faltan algunos (Age) o faltan prácticamente todos (Cabin). Luego, será necesario aplicar métodos distintos en cada caso.

####  Embarked

In [ ]:
# sólo nos faltan dos valores: veamos como están distribuídos los datos
train_df.groupby('Embarked'). count().PassengerId

La mayoría de los pasajeros embarcaron en el puerto S, por lo tanto parece razonable inputar este valor:

In [ ]:
train_df = train_df.fillna({'Embarked' : 'S'})

#### Age

Al tratarse de una variable continua con un mayor número de valores faltantes hace falta ir con un poco más de cuidado. Aún así, una primera aproximación fácil sería inputar como valor la media de las edades:

In [ ]:
mean_age = train_df['Age'].mean()
train_df = train_df.fillna({'Age' : mean_age})

Aún así se trata de una técnica muy burda, ya que nos estamos inventando muy *grosso modo* 177 valores!

#### Cabin

Al no tener prácticamente ningún valor en este atributo, no perdemos apenas información si decidimos eliminar toda la columna:

In [ ]:
train_df.drop(['Cabin'], axis = 1, inplace = True)

In [ ]:
# Comprovamos que realmente se han eliminado todos los nulos:
print pd.isnull(train_df).sum(), '\n'

Tener en cuenta que cualquier cambio que apliquemos al train dataset debe hacerse sobre el test, y más cuando estamos hablando de valores nulos. Ya hemos visto que allí sólo faltan valores en las columnas de *Age* y *Cabin*, por lo tanto no hace falta aplicar ningún cambio en *Embarked*.

In [ ]:
mean_age2 = test_df['Age'].mean()
test_df = test_df.fillna({'Age' : mean_age2})

test_df.drop(['Cabin'], axis = 1, inplace = True)

Aún así queda por arreglar el valor nulo de *Fare*. Usemos el mismo criterio que con la edad:

In [ ]:
mean_fate = test_df.Fare.mean()
test_df = test_df.fillna({'Fare' : mean_fate})

Si nos fijamos en la columna nombre, actua como si de un ID se tratase: vamos a eliminarlo también de ambos datasets. También vamos a eliminar la columna *Ticket* ya que a primera vista no parece que aporte ningún tipo de información útil. Para poder subir nuestra predicción a la plataforma *Kaggle* debemos guardarnos el id del test.

In [ ]:
train_df.drop(['Name', 'Ticket','PassengerId'], axis = 1, inplace = True)
test_id = test_df.PassengerId
test_df.drop(['Name', 'Ticket','PassengerId'], axis = 1, inplace = True)

## Predicción: aplicación de un modelo 

LLega el momento de utilizar algoritmos para predecir la (no) supervivencia de nuestros pasajeros a bordo del titanic. Una libreria muy interesante de *python* en el ámbito del *Machine Learning* es **scikit-learn (sklearn)**. Esta librería contiene muchos algoritmos ya implementados, así como módulos para aplicar cambios necesarios en nuestros datos de forma más sencilla. Realizaremos la predicción usando diferentes algoritmos:

* Gaussian Naive Bayes
* Logistic Regression
* Support Vector Machine (SVM)
* Perceptron
* Decision Tree Classifier
* Random forest Classifier
* K-Nearest Neighbors
* Stochastic Gradient Descent
* Gradient Boosting Classifier

Antes de ponernos a utilizar lo modelos, usaremos un par de módulos de sklearn para arreglar 2 cosas:
1. Estos algoritmos no admiten variables categóricas de tipo string. Solo admiten valores enteros o decimales. Por ello usaremos un *label encoder*. Lo que hace este label encoder básicamente es usar un diccionario para pasar nuestras variables categóricas a categorías numéricas.
2. Dividiremos los datos en **entrenamiento** y **validación**. Esto es muy importante, ya que para calcular la precisión del modelo necesitamos unos datos para comprobar nuestras predicciones. Habitualmente, se coge un 70/80% de los datos para *entrenar* el modelo, y un 20/30% para *validar* los resultados

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Parte 1: Codificar las variables categóricas
# Creamos un clasificador para cada variable categórica a codificar. Podemos crear solo uno para todas,
# pero entonces no nos guarda las categorías por si queremos invertir la codificación.
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

# La manera más rapida:
train_df.Sex = le1.fit_transform(train_df.Sex)
test_df.Sex = le1.transform(test_df.Sex)

train_df.Embarked = le2.fit_transform(train_df.Embarked)
test_df.Embarked = le2.transform(test_df.Embarked)

# Parte 2: Dividir los datos en training y testing:
predictors = train_df.drop(['Survived'], axis=1)
target = train_df["Survived"]

# fijando el valor de random_state nos aseguramos que aunque volvamos a córrer el código, la partición se mantenga igual.
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.22, random_state = 12)

Llega el momento de aplicar los diferentes modelos a nuestros datos y ver qué resultados obtenemos usando la librería sklearn de python.

Antes que nada , notar que todos los algoritmos mencionados son **Algoritmos de Clasificación, no de Regresión**.

Cómo ejemplo vamos a usar el **Gaussian Naive Bayes**. Este algoritmo de aprendizaje supervisado trabaja con probabilidades, usando el *Teorema de Bayes*. Calcula la probabilidad de que un elemento esté en cada una de las clases que tenemos. La clase con mayor probabilidad es la que se devuelve como predicción. Este algoritmo asume que las direfentes variables no están relacionadas entre sí, que todas contribuyen de igual manera al resultado de la clasificación.

(En otro notebook a parte encontraréis la explicación y la implementación de otros modelos).

In [ ]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
acc_gau = round(accuracy_score(y_pred, y_val) * 100, 2)
print 'Precisión del algoritmo de clasificación \"Gaussian Naive Bayes\": {0} % '.format(acc_gau)

Ahora ya podemos aplicar el modelo a nuestros datos de test:

In [ ]:
y_test = gaussian.predict(test_df)

Nos guardamos el resultado en un Dataframe a parte que vamos a escribir en forma de archivo csv. Éste se guardará en el directorio donde tengamos el notebook o bien el que explicitemos.

In [ ]:
result = pd.DataFrame({'PassengerId': test_id,'Survived': y_test}, columns=['PassengerId','Survived'])
result.head()

In [ ]:
result.to_csv('first_submission.csv', sep=",", index=False)

### Submit results

Para poder colgar el resultado a la plataforma, hacer entrar en: https://www.kaggle.com/c/titanic y entrar en *Make Submission*, donde subiremos el archivo que acabamos de generar y se nos devolverá un *score* indicando la precisión en las predicciones.

Tener en cuenta que cada cuenta sólo podrá hacer submit de 5 pruebas al día.